In [ ]:
import pandas as pd
import numpy as np
from combine_data import combine_data
from data_processing import ADC_to_v,format_timestamps,normalize_data
import librosa
import os,shutil
import matplotlib.pyplot as plt
import scipy
from scipy.stats import kurtosis,skew

In [ ]:


raw_data,labels,time_stamps = combine_data('./../data/al_data')
labels -=1

In [ ]:
voltage_values = []
time = []
for i in raw_data:
    voltage_values.append(ADC_to_v(i))
for i in time_stamps:
    time.append(format_timestamps(i))

In [ ]:
n_sample = 7

In [ ]:
win_size = 200
# plt.subplot(3,1,1)
f1 = np.lib.stride_tricks.sliding_window_view(voltage_values[n_sample],win_size).mean(axis=1)
# f2 = skewness(np.lib.stride_tricks.sliding_window_view(voltage_values[n_sample],win_size),axis=1)/f3
f3 = np.lib.stride_tricks.sliding_window_view(voltage_values[n_sample],win_size).max(axis=1)
f4 = kurtosis(np.lib.stride_tricks.sliding_window_view(voltage_values[n_sample],win_size),axis=1)
f2 = skew(np.lib.stride_tricks.sliding_window_view(voltage_values[n_sample],win_size),axis=1)
f3 = np.apply_along_axis(lambda x: x*(x>0),0,f4*f2 > f3)
plt.plot(voltage_values[n_sample])
# plt.plot(f1)
# plt.plot(f2)
plt.plot(f3)
# plt.plot(f4)
plt.grid()

# plt.subplot(3,1,2)
# plt.plot(voltage_values[n_sample+1])
# plt.plot(np.convolve(voltage_values[n_sample+1],np.ones(win_size)/win_size,'valid'))
# plt.plot(np.lib.stride_tricks.sliding_window_view(abs(voltage_values[n_sample+1]),win_size).mean(axis=1))
# plt.plot(np.lib.stride_tricks.sliding_window_view(voltage_values[n_sample+1],win_size).max(axis=1))
# plt.plot(np.lib.stride_tricks.sliding_window_view(voltage_values[n_sample+1],win_size).min(axis=1))
# plt.grid()
# plt.subplot(3,1,3)
# plt.plot(voltage_values[n_sample+2])
# plt.plot(np.convolve(voltage_values[n_sample+2],np.ones(win_size)/win_size,'valid'))
# plt.plot(np.lib.stride_tricks.sliding_window_view(abs(voltage_values[n_sample+2]),win_size).mean(axis=1))
# plt.plot(np.lib.stride_tricks.sliding_window_view(voltage_values[n_sample+2],win_size).max(axis=1))
# plt.plot(np.lib.stride_tricks.sliding_window_view(voltage_values[n_sample+2],win_size).min(axis=1))
# plt.grid()

In [ ]:
np.lib.stride_tricks.sliding_window_view(voltage_values[n_sample],250,).var(axis=1)

In [ ]:
from librosa import display

In [ ]:
from scipy.signal import butter, lfilter


def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def filter(voltage_values,low_cut=1,high_cut=55,fs = 1e3,):
    '''
    band pass filter based on the scipy butter bandpass, allowing frequency between low cut and high cut to pass through.
    input : np.array of the data voltage values.
    low_cut : lowest frequency defaults at 1
    high cut : highest frequency in the band filter defaults at 55.
    '''
    return butter_bandpass_filter(voltage_values,lowcut=low_cut,highcut=high_cut,fs = fs)

In [ ]:
from scipy.signal import butter, lfilter


def butter_bandpass(lowcut, highcut, fs, order=5):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
from scipy.fft import fft, fftfreq
fft_vals = abs(fft(voltage_values[n_sample],n=16))
fft_vals

In [ ]:
plt.plot(librosa.fft_frequencies(sr=1000, n_fft=16),fft_vals[:len(fft_vals)//2+1])

In [ ]:
filtered = butter_bandpass_filter(voltage_values[n_sample],1,40,1000)

In [ ]:
plt.plot(filtered)
plt.grid()

In [ ]:
from scipy.integrate import cumtrapz

In [ ]:
plt.plot((filtered))
plt.plot(1/np.pi * cumtrapz(filtered))

In [ ]:
from feature_extraction import *

In [ ]:
filtered_values = []
for i in voltage_values:
    filtered_values.append(filter(i,high_cut=60))

In [ ]:

plt.subplot(1,2,1)
plt.title(f"{labels[n_sample]} filtered")
plt.plot(filtered)
plt.grid()
plt.subplot(1,2,2)
plt.title(f"{labels[n_sample]} raw")
plt.plot(voltage_values[n_sample])
plt.grid()

In [ ]:
features = [root_mean_squared,iemg,mean_absolute_value,variance,wave_form_length,mean,median,skewness,kurt,zero_crossing_rate]
df = extract_features(voltage_values,label = labels,features_no=10,overlapping_percentage=0.25,features_funcs=features)
df

In [ ]:
df.columns

In [ ]:
import seaborn as sns
from sklearn.decomposition import PCA
import pickle as pk


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale,normalize,StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import Sequential

In [ ]:
X,y = df.drop('label',axis = 1),df['label']
X_scaled =minmax_scale(X)

In [ ]:
pca = PCA(n_components=25)
pca.fit(X_scaled)
components = pca.transform(X_scaled)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(components,y,train_size=0.7)
x_train.shape

In [ ]:
with open('./pca.pkl','wb') as pkl:
    pk.dump(pca,pkl)

In [ ]:
# parameter 1 
encoding_dim = 100
# ncol # without the label column
ncol = x_train.shape[1:][0]
ncol

In [ ]:
model = Sequential()
model.add(keras.layers.Input(shape=x_train.shape[1:]))
model.add(keras.layers.Dense(15))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(5))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(3,activation = 'softmax'))

In [ ]:
loss_fn = loss_fn = keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),loss=loss_fn,metrics = ['accuracy'],)

In [ ]:
model.summary()

In [ ]:
model.fit(x = x_train,y=y_train,validation_split=0.2,epochs=200,verbose=False)

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC()
svm.fit(x_train,y_train)

In [ ]:
predictions_ann = np.argmax(model.predict(x_test),axis=1)
predictions_svc = svm.predict(x_test)
from sklearn.metrics import classification_report
print(classification_report(y_true = y_test,y_pred=predictions_ann))

In [ ]:
model.save('ann.h5')

In [ ]:
import inspect

In [ ]:
print(inspect.getsource(ADC_to_v))

In [ ]:
dataMax = np.max([i for i in map(len,raw_data)])
out_size = 128
max_features = 20000

In [ ]:
X = np.array(raw_data)
y = np.array(labels)

In [ ]:
for i,element in enumerate(X):
    temp = np.zeros(dataMax)
    temp[:len(element)] += element[0]
    X[i] = np.array(temp,dtype='float32')
    

In [ ]:
X = tf.convert_to_tensor([tf.convert_to_tensor(i) for i in X])

In [ ]:
X = np.expand_dims(X,axis=2)

In [ ]:
model = Sequential()
model.add(keras.layers.Input(shape = X.shape[1:]))
model.add(keras.layers.Masking(mask_value=0))
model.add(keras.layers.Conv1D(2,))
model.add(keras.layers.Dense(3,activation = 'softmax'))
loss_fn = loss_fn = keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),loss=loss_fn,metrics = ['accuracy'],)

In [ ]:
model.build()

In [ ]:
model.summary()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y)

In [ ]:
model.fit(x_train,y_train)

In [ ]:
import os
import pandas as pd
import numpy as np
import re
def combine_data(path:str):
    """
    this script will read the json files and structures it itno X,y for preprocessing and feature extraction.
    the input is the son data path.
    this function will outputs 3 np.array tuples raw data, labels, timestamps.
    """
    data = []
    labels = []
    time_stamps = []
    for file_name in os.listdir(path):
        if file_name.endswith("json"):
            df = pd.read_json(path + '/'+file_name)
            data.append(np.asarray(df.data.values,dtype=np.float32))
            #labels.append(df.label.unique()[0])
            labels.append(re.findall(pattern=r'[A-Z]\w+',string=file_name)[0])
            #time_stamps.append(df.time)
    data = np.array(data)
    labels = np.array(labels)
    time_stamps = np.array(time_stamps)
    return data,labels,time_stamps

In [17]:
data,labels,time_st = combine_data(r'C:\Users\aleja\Documents\Courses\Applied_AI\emg_DVA435_CDT406\data\naz_data')

In [ ]:
#df_naz = df = pd.DataFrame(data)
def sliding_window(data, window_size, overlap):
    """
    Applies a sliding window with overlap to the input data.

    Parameters:
    data (ndarray): input data
    window_size (int): size of sliding window
    overlap (int): overlap between consecutive windows

    Returns:
    ndarray: 2D array with sliding windows of shape (n_windows, window_size)
    """
    n_samples = data.shape[0]
    overlap = int(np.floor((window_size*overlap)/100))
    n_windows = int(np.floor((n_samples - window_size) / overlap) + 1)
    windows = np.zeros((n_windows, window_size))

    for i in range(n_windows):
        start = i * overlap
        end = start + window_size
        windows[i] = data[start:end]

    return windows

def extract_features(data, feature_names):
    """
    Extracts features from data and returns the fitted model object.
    :param data: data with signals
    :param overlap: overlaping space
    """
    # Work with numpy matrix
    data = np.array(data)

    # Time domain features: Mean absolute value (MAV), root mean square (RMS), waveform length (WL),zero crossings (ZC), variance (VAR)
    '''feature_names = ['Variance(VAR)', 'MeanAbsoluteValue(MAV)', 'RootMeanSquare(RMS)', 'WaveformLength(WL)',
                     'Mean(MEAN)',
                     'StandardDeviation(SD)', 'Median', 'Peak(PK)', 'Min(MIN)', 'IntegralEMG(iEMG)',
                     'AverageAmplitudeChange(ACC)',
                     'Kurtosis(KURT)', 'Skewness(SKEW)']'''
    '''var = []
    mav = []
    rms = []
    wl = []
    zc = []
    mean = []
    std = []
    median = []
    peak = []
    min = []
    iemg = []
    aac = []
    kur = []
    skewe = []'''

    # Iterate for each signal/window and calculate associated features
    '''for x in range(len(data_n)):'''
    var = np.var(data) 
    rms = np.sqrt(np.mean(data ** 2))
    mav = np.sum(np.absolute(data)) / len(data)
    wl = np.sum(abs(np.diff(data)))
    mean = np.mean(data)
    std = np.std(data)
    median = np.median(data)
    peak = np.max(data)
    min = np.min(data)
    iemg = np.sum(abs(data))
    aac = np.sum(abs(np.diff(data))) / len(data)
    kur = kurtosis(data)
    skewe = skew(data)

        # zc.append(zcruce(x, th))

    matrix = np.column_stack((var, mav, rms, wl, mean, std, median, peak, min, iemg, aac, kur, skewe))
    df_features = pd.DataFrame(matrix, columns=feature_names)
    #data = pd.concat([data, df_features], axis=1)

    return df_features

In [9]:
data[0:3]

array([[-4., -4., -4., ..., -3., -2., -6.],
       [-4., -4., -6., ...,  6.,  4.,  5.],
       [ 1.,  1.,  0., ..., -6., -7., -9.]], dtype=float32)

In [21]:
from scipy.stats import skew, kurtosis
col_names = ['Variance(VAR)', 'MeanAbsoluteValue(MAV)', 'RootMeanSquare(RMS)', 'WaveformLength(WL)',
                     'Mean(MEAN)',
                     'StandardDeviation(SD)', 'Median', 'Peak(PK)', 'Min(MIN)', 'IntegralEMG(iEMG)',
                     'AverageAmplitudeChange(ACC)',
                     'Kurtosis(KURT)', 'Skewness(SKEW)']

window_sizes = [250]
overlaps = [50]
frames = data
df_naz = pd.DataFrame()

for window_size in range(len(window_sizes)):
  for overlap in range(len(overlaps)):
    for frame in range(len(frames)):
      windows = sliding_window(frames[frame], window_sizes[window_size], overlaps[overlap])
      df_row = pd.DataFrame()
      for window in range(len(windows)):
        current_col_names = ["%s_%s" % (col_names[c], str(window)) for c in range(len(col_names))]
        window_features = extract_features(windows[window], current_col_names)
        df_row = pd.concat([df_row, window_features], axis=1)
      df_naz = pd.concat([df_naz, df_row], axis=0)
      

In [24]:
df_naz['label'] = labels

In [26]:
df_naz.to_csv(r'C:\Users\aleja\Documents\Courses\Applied_AI\emg_DVA435_CDT406\data\naz_data\df_naz.csv', index=False)